# pyBrep Description and to do list

**Dear soon-to-be pyBrep enthusiast!**

This notebook is supposed to help you get started with, use and further develope pyBREP. It contains
- A quick overview over the structure
- A documentation of the material that I used while developing that could possibly be helpful
- Differences to the original (scheme) BREP
- A to-do list containing known caveats and more nice-to-have things (and, if available, my initial ideas on how to tackle them)

If you have any question, do not hesitate to contact me, I am happy to try and help! 

Best, Ines

## Short overview over the structure of pyBREP

pyBREP consists essentially of the following components:
- **The population files**: So far there are only Golgi and Granule cell populations. Those populations contain routines to read in data from coordinate files, render cell structures (like dendrites or axons) by representing them with points or lines, to save those cell structures and to prepare so-called query points.
- **Query_point** is a small class that contains the coordinates of the data points from one population, as well as other information (for example, each Golgi cell has a lot of points that represent the dendrites

## List of to dos and nice-to-haves:

### Population generation (caveat)
Right now, the coordinates of the populations are generated from the simulation and pyBrep just reads them in.
However, it seems as if those coordinates are 'not random enough': If you plot them as projections to the x-y or the y-z axis, you can see that there are 'stripes' in the z axis. With pyBrep, this leads to 

### Test the code with more dendrites
I generally tried to make it possible that there is an arbitrary number of dendrites. But I did not really evaluate that. Sungho mentioned he might want to add more dendrites than there are currently existing.

### Multiple connections (will probably be fixed by Sungho in the big simulation)

### Test varying aa length (talk to Sungho, that is probably more a new project...)

## On the parameters.hoc file...
PFzoffset
- Dendrite_theta_min/max: They are actually just angle 1 and 2